In [15]:
# Load libraries
import pandas as pd
import glob
import os
import numpy as np

In [ ]:
# Define the folder path containing the Excel files
folder_path = "../../data/instat/"

# Pattern matching Excel files starting with "Games" and ending with ".xlsx"
file_pattern = os.path.join(folder_path, "Games*.xlsx")
files = glob.glob(file_pattern)

print("Found files:")
for f in files:
    print(f)

# Define the mapping from part of the filename to game type label
type_mapping = {
    "TotalOpp": "Total",
    "EVOpp": "Even Strength",
    "PPOpp": "Power Play",
    "PKOpp": "Penalty Kill"
}

# List to collect DataFrames after processing each file
dfs = []

# Specify the columns with merged cells.
merged_cols = ['Date', 'Opponent', 'Score', 'Penalty time']

# Process each file
for file in files:
    # Read the file using the openpyxl engine
    df = pd.read_excel(file, engine='openpyxl')
    print(f"\nProcessing file: {file}")
    print("Original DataFrame:")
    print(df.head())

    # Replace placeholder dashes '-' with NA in the merged columns
    df[merged_cols] = df[merged_cols].replace('-', pd.NA)

    # Forward-fill the Date, Opponent, and Score columns so that each game group has the same info
    df[['Date', 'Opponent', 'Score']] = df[['Date', 'Opponent', 'Score']].fillna(method='ffill')

    # For the "Penalty time" column, if the current row belongs to the same game (i.e. Date, Opponent, Score match),
    # then override its value with the previous row's value.
    for i in range(1, len(df)):
        if df.loc[i, ['Date', 'Opponent', 'Score']].equals(df.loc[i-1, ['Date', 'Opponent', 'Score']]):
            df.loc[i, 'Penalty time'] = df.loc[i-1, 'Penalty time']

    # Optionally remove rows that might represent "average per game" entries.
    avg_pattern = r'(?i)avg|average'
    mask = df[merged_cols].apply(lambda col: col.astype(str).str.contains(avg_pattern, na=False))
    df = df[~mask.any(axis=1)]

    # Infer the game type from the filename.
    base_name = os.path.basename(file)
    game_type = None
    for key, label in type_mapping.items():
        if key in base_name:
            game_type = label
            break
    # Fallback if no matching key is found.
    if game_type is None:
        game_type = "Unknown"

    # Add a new column indicating what the file represents
    df['Type'] = game_type

    # Append the processed DataFrame to our list
    dfs.append(df)

Found files:
../../data/instat\GamesEVOpp.xlsx
../../data/instat\GamesPKOpp.xlsx
../../data/instat\GamesPPOpp.xlsx
../../data/instat\GamesTotalOpp.xlsx

Processing file: ../../data/instat\GamesEVOpp.xlsx
Original DataFrame:
    Date                       Opponent Score Unnamed: 3  Goals  Penalties  \
0  22/02              @ Neumann Knights   4:2        HCB    3.0        5.0   
1      -                              -     -         NK    0.0        4.0   
2  21/02             vs Wilkes Colonels   1:7        HCB    1.0        1.0   
3      -                              -     -         WC    6.0        2.0   
4  14/02  vs Arcadia University Knights   1:4        HCB    1.0        3.0   

   Penalties drawn Penalty time  Faceoffs  Faceoffs won  ...  \
0              4.0        10:00        40            22  ...   
1              5.0        08:00        40            18  ...   
2              2.0        02:00        56            23  ...   
3              1.0        04:00        56          

C:\Users\03dng\AppData\Local\Temp\ipykernel_34168\1246365661.py:42: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[['Date', 'Opponent', 'Score']] = df[['Date', 'Opponent', 'Score']].fillna(method='ffill')
C:\Users\03dng\AppData\Local\Temp\ipykernel_34168\1246365661.py:42: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[['Date', 'Opponent', 'Score']] = df[['Date', 'Opponent', 'Score']].fillna(method='ffill')
C:\Users\03dng\AppData\Local\Temp\ipykernel_34168\1246365661.py:42: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[['Date', 'Opponent', 'Score']] = df[['Date', 'Opponent', 'Score']].fillna(method='ffill')



Processing file: ../../data/instat\GamesPKOpp.xlsx
Original DataFrame:
    Date                       Opponent Score Unnamed: 3  Goals  Penalties  \
0  22/02              @ Neumann Knights   4:2        HCB    0.0        1.0   
1      -                              -     -         NK    0.0        0.0   
2  21/02             vs Wilkes Colonels   1:7        HCB    0.0        1.0   
3      -                              -     -         WC    0.0        0.0   
4  14/02  vs Arcadia University Knights   1:4        HCB    0.0        0.0   

   Penalties drawn Penalty time  Faceoffs  Faceoffs won  ...  \
0              0.0        02:00      17.0           6.0  ...   
1              0.0        00:00       4.0           0.0  ...   
2              0.0        02:00       2.0           1.0  ...   
3              1.0        00:00       6.0           4.0  ...   
4              0.0        00:00       7.0           1.0  ...   

  Loose puck recovery  Opponent’s dump-in retrievals  Entries  \
0        

C:\Users\03dng\AppData\Local\Temp\ipykernel_34168\1246365661.py:42: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[['Date', 'Opponent', 'Score']] = df[['Date', 'Opponent', 'Score']].fillna(method='ffill')


### Concatenate all processed DataFrames into a single DataFrame.

In [6]:
games_opps_df = pd.concat(dfs, ignore_index=True)

print("\nConcatenated DataFrame:")
print(games_opps_df.head())


Concatenated DataFrame:
    Date                       Opponent Score Unnamed: 3  Goals  Penalties  \
0  22/02              @ Neumann Knights   4:2        HCB    3.0        5.0   
1  22/02              @ Neumann Knights   4:2         NK    0.0        4.0   
2  21/02             vs Wilkes Colonels   1:7        HCB    1.0        1.0   
3  21/02             vs Wilkes Colonels   1:7         WC    6.0        2.0   
4  14/02  vs Arcadia University Knights   1:4        HCB    1.0        3.0   

   Penalties drawn Penalty time  Faceoffs  Faceoffs won  ...  \
0              4.0        10:00      40.0          22.0  ...   
1              5.0        10:00      40.0          18.0  ...   
2              2.0        02:00      56.0          23.0  ...   
3              1.0        02:00      56.0          33.0  ...   
4              3.0        06:00      40.0          17.0  ...   

  Opponent’s dump-in retrievals  Entries  Entries via pass  \
0                          15.0     35.0              13.0 

Optionally, save the concatenated DataFrame to a new Excel or CSV file:

In [7]:

games_opps_df.to_excel("GamesOppCombined.xlsx", index=False)
# games_opps_df.to_csv("GamesOppCombined.csv", index=False)


In [16]:
games_opps_df = pd.read_excel("GamesOppCombined.xlsx")

## Data Cleaning

### NAs

In [17]:
# Replace '-' with np.nan for consistency in identifying null values
games_opps_df.replace('-', np.nan, inplace=True)

C:\Users\03dng\AppData\Local\Temp\ipykernel_28704\1726508973.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  games_opps_df.replace('-', np.nan, inplace=True)


In [18]:
# Count the number of null values in each column
null_counts = games_opps_df.isnull().sum()

# Display columns with null values and their counts
null_info = null_counts[null_counts > 0]
print("Columns with null values and their counts:")
print(null_info)

Columns with null values and their counts:
Faceoffs won, %                   7
Faceoffs won in DZ, %            44
Faceoffs won in NZ, %            67
Faceoffs won in OZ, %            47
CORSI%                           96
Power play                       96
Successful power play            96
Power play time                  96
Power play, %                   154
Short-handed                     96
Penalty killing                  96
Short-handed time                96
Short-handed, %                  98
% shots on goal                  26
Power play shots                 96
Short-handed shots               96
Puck battles won, %               2
Dekes successful, %              62
Accurate passes, %                8
OZ play with shots, %            47
Counter-attack with shots, %     84
EV OZ retrievals                 96
EV DZ retrievals                 96
dtype: int64


In [19]:
# Replace NAs in time-related columns with "0:00"
time_columns = [col for col in games_opps_df.columns if 'time' in col.lower()]
games_opps_df[time_columns] = games_opps_df[time_columns].fillna("0:00")

# Replace NAs in percentage-related columns with "0%"
percentage_columns = [col for col in games_opps_df.columns if '%' in col]
games_opps_df[percentage_columns] = games_opps_df[percentage_columns].fillna("0%")

# Replace all remaining NAs with 0
games_opps_df.fillna(0, inplace=True)

In [ ]:
from IPython.display import display, HTML

# Get the data types of the columns in the DataFrame
column_dtypes = games_opps_df.dtypes

# Convert the data types Series to a DataFrame for better formatting
column_dtypes_df = column_dtypes.reset_index()
column_dtypes_df.columns = ['Column', 'Data Type']

# Display the DataFrame as a scrollable HTML table
display(HTML(column_dtypes_df[column_dtypes_df['Data Type'] == 'object'].to_html(index=False, max_rows=None, max_cols=None)))

Column,Data Type
Date,object
Opponent,object
Score,object
Unnamed: 3,object
Penalty time,object
"Faceoffs won, %",object
"Faceoffs won in DZ, %",object
"Faceoffs won in NZ, %",object
"Faceoffs won in OZ, %",object
CORSI%,object


In [14]:
# Remove '%' and convert percentage columns to numeric
for col in percentage_columns:
    games_opps_df[col] = games_opps_df[col].str.rstrip('%').astype(float) * 0.01

print("Percentage columns converted to numeric:")
print(games_opps_df[percentage_columns].dtypes)

AttributeError: Can only use .str accessor with string values!

In [11]:
# Display percentage columns after conversion
print(games_opps_df[percentage_columns].head())

   Faceoffs won, %  Faceoffs won in DZ, %  Faceoffs won in NZ, %  \
0             0.55                   0.38                   0.75   
1             0.45                   0.47                   0.25   
2             0.41                   0.43                   0.28   
3             0.59                   0.38                   0.72   
4             0.43                   0.42                   0.33   

   Faceoffs won in OZ, %  CORSI%  Power play, %  Short-handed, %  \
0                   0.53    0.62            0.0              0.0   
1                   0.62    0.38            0.0              0.0   
2                   0.63    0.14            0.0              0.0   
3                   0.57    0.86            0.0              0.0   
4                   0.75    0.17            0.0              0.0   

   % shots on goal  Puck battles won, %  Dekes successful, %  \
0             0.58                 0.57                 0.69   
1             0.54                 0.43               

In [12]:
non_numeric_info = column_dtypes[column_dtypes == 'object']
print("Non-numeric columns and their data types:")
print(non_numeric_info)

Non-numeric columns and their data types:
Date                            object
Opponent                        object
Score                           object
Unnamed: 3                      object
Penalty time                    object
Faceoffs won, %                 object
Faceoffs won in DZ, %           object
Faceoffs won in NZ, %           object
Faceoffs won in OZ, %           object
CORSI%                          object
Power play time                 object
Power play, %                   object
Short-handed time               object
Short-handed, %                 object
% shots on goal                 object
Offensive play                  object
Defensive play                  object
OZ possession                   object
NZ possession                   object
DZ possession                   object
Puck battles won, %             object
Dekes successful, %             object
Accurate passes, %              object
OZ play with shots, %           object
Counter-attack with sh

In [79]:
# Display non-numeric columns
non_numeric_columns_df = games_opps_df[non_numeric_columns]
non_numeric_columns_df.head()

,Date,Opponent,Score,Unnamed: 3,Penalty time,Power play time,Short-handed time,Offensive play,Defensive play,OZ possession,NZ possession,DZ possession,Type
0,22/02,@ Neumann Knights,4:2,HCB,10:00,0:00,0:00,21:25,16:16,11:03,04:28,05:54,Even Strength
1,22/02,@ Neumann Knights,4:2,NK,10:00,0:00,0:00,16:16,21:25,05:31,03:45,07:00,Even Strength
2,21/02,vs Wilkes Colonels,1:7,HCB,02:00,0:00,0:00,15:25,32:11,02:29,03:07,09:50,Even Strength
3,21/02,vs Wilkes Colonels,1:7,WC,02:00,0:00,0:00,32:11,15:25,19:54,05:12,07:04,Even Strength
4,14/02,vs Arcadia University Knights,1:4,HCB,06:00,0:00,0:00,17:32,23:24,03:42,03:50,10:01,Even Strength


## Data Wrangling

In [68]:
games_opps_df.head()

,Date,Opponent,Score,Unnamed: 3,Goals,Penalties,Penalties drawn,Penalty time,Faceoffs,Faceoffs won,...,Opponent’s dump-in retrievals,Entries,Entries via pass,Entries via dump in,Entries via stickhandling,Breakouts,Breakouts via pass,Breakouts via dump out,Breakouts via stickhandling,Type
0,22/02,@ Neumann Knights,4:2,HCB,3,5,4,1900-01-01 00:10:00,40,22,...,15,35,13,4,18,35,19,0,16,Even Strength
1,22/02,@ Neumann Knights,4:2,NK,0,4,5,1900-01-01 00:10:00,40,18,...,15,33,10,1,22,26,16,1,9,Even Strength
2,21/02,vs Wilkes Colonels,1:7,HCB,1,1,2,1900-01-01 00:02:00,56,23,...,10,20,3,1,16,31,11,3,17,Even Strength
3,21/02,vs Wilkes Colonels,1:7,WC,6,2,1,1900-01-01 00:02:00,56,33,...,10,47,13,6,28,30,17,1,12,Even Strength
4,14/02,vs Arcadia University Knights,1:4,HCB,1,3,3,1900-01-01 00:06:00,40,17,...,14,19,4,4,11,37,18,1,18,Even Strength


In [69]:
games_opps_df['isOpponent'] = games_opps_df['Unnamed: 3'] != 'HCB'
print(games_opps_df[['Unnamed: 3', 'isOpponent']].head())

  Unnamed: 3  isOpponent
0        HCB       False
1         NK        True
2        HCB       False
3         WC        True
4        HCB       False


In [70]:
# Ensure the 'Opponent' column is treated as string
games_opps_df['Opponent'] = games_opps_df['Opponent'].astype(str)

# Determine if the game is away based on '@' symbol
contains_at = games_opps_df['Opponent'].str.contains('@')
is_opponent = games_opps_df['isOpponent']

# Calculate the 'isAway' status
# If isOpponent is True, flip the away status (True if 'vs', False if '@')
# If isOpponent is False, use the original away status (True if '@', False if 'vs')
games_opps_df['isAway'] = np.where(is_opponent, ~contains_at, contains_at)

# Extract the opponent name
# Use regex to capture text after '@\n' or 'vs\n'
extracted_name = games_opps_df['Opponent'].str.extract(r'[(?:@|vs)\n*](.*)')[0]

# Update the 'Opponent' column
# If isOpponent is True, set Opponent to 'Hood'
# If isOpponent is False, set Opponent to the extracted name
games_opps_df['Opponent'] = np.where(is_opponent, 'Hood', extracted_name)

# Display the relevant columns to verify
print(games_opps_df[['Opponent', 'isAway', 'isOpponent']].head())

                       Opponent  isAway  isOpponent
0               Neumann Knights    True       False
1                          Hood   False        True
2             s Wilkes Colonels   False       False
3                          Hood    True        True
4  s Arcadia University Knights   False       False
